In [1]:
# Google Colab 환경에서 Google Drive 파일을 사용하기 위한 설정
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFa967z-UKIH6j0SuMjHa5bAD25eYCCV-2hNBos0DDiGzM4xQ3NeBM
Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
import graphviz
import pydot
from datetime import datetime
from datetime import timedelta
from datetime import date

In [3]:
# 필요한 데이터 탑재
df_activity_info = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/딥스_ACTIVITY.csv", encoding='utf-8')
# df_assy_activity = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/딥스_ASSYACTIVITY.csv", encoding='utf-8')
# df_pe_activity = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/딥스_PEACTIVITY.csv", encoding='utf-8')
# df_relation = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/딥스_RELATION.csv", encoding='utf-8')
# df_pe_sequence = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/딥스_PESEQUENCE.csv", encoding='utf-8')
# df_erection_sequence = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/딥스_ERECTIONSEQUENCE.csv", encoding='utf-8')
df_activity_info = df_activity_info[df_activity_info.columns[:25]]

In [4]:
df_activity = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/ACTIVITY.csv", encoding='utf-8')
df_relation = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/DSME Network/data/RELATION.csv", encoding='utf-8')
df_activity.drop(df_activity.columns[-1], axis=1, inplace=True)
df_relation.dropna(how='all', inplace=True)

In [5]:
# ACTIVITYCODE -> 공정 종류에 해당하는 KEY 생성에 쓰일 column list 미리 선언
id_list = ['FLAGCODE', 'PROCESSCODE', 'PROCESSSERNO']
# 계획 및 실적 착수/완료일 데이터 전처리를 용이하게 하기 위해 column list 미리 선언
date_col_list = ['PLANSTARTDATE', 'PLANFINISHDATE', 'ACTUALSTARTDATE', 'ACTUALFINISHDATE']

In [6]:
# activity dataframe 확인
df_activity

,PROJECTNO,ACTIVITYCODE,LOCATIONCODE,LOCATIONTYPE,DESCRIPTION,PLANDURATION,물량,단위,작업장
0,5466,P571BP00,571,B,571 BLK B Stage 선행 도장,5,561,M²,선행도장-상선-도장
1,5466,W571GA00,571,B,571 BLK A Stage 대조 의장 ('A'),14,1419.6,KG,조립의장-상선관철
2,5466,S634A000,634,B,634 BLK B Stage 조립,15,2641,JL,조립1-PBS
3,5466,C621CC00,621,B,621 BLK J Stage 절단,18,98.6,TON,가공-상선-절단
4,5466,S426A000,426,B,426 BLK B Stage 조립,9,624.1,JL,조립3-상선-NPS
...,...,...,...,...,...,...,...,...,...
2767,5466,E20KP50K,50K,New Activity,20K-50K PE Seq Act,1,-,-,HZ-2 PE
2768,5466,E20LP50L,50L,New Activity,20L-50L PE Seq Act,1,-,-,No.1 Dock Goaliath Crane
2769,5466,E20MP50M,50M,New Activity,20M-50M PE Seq Act,1,-,-,No.1 Dock Goaliath Crane
2770,5466,E20NP50N,50N,New Activity,20N-50N PE Seq Act,1,-,-,No.1 Dock Goaliath Crane


In [7]:
# relation dataframe 확인
df_relation

,COMPANYID,RELATIONNO,CASENO,PROJECTNO,PREVACTIVITYCODE,NEXTACTIVITYCODE,RELATIONTYPE,LAG,LOCATIONCODE,LOCATIONTYPE
0,D,1.114171e+09,2020-36-04,5466.0,S101A000,C101CC00,SS,-7.0,101,B
1,D,1.114171e+09,2020-36-04,5466.0,C101CC00,C101CP00,SS,-1.0,101,B
2,D,1.114171e+09,2020-36-04,5466.0,S102A000,C102CC00,SS,-7.0,102,B
3,D,1.114171e+09,2020-36-04,5466.0,C102CC00,C102CP00,SS,-1.0,102,B
4,D,1.114171e+09,2020-36-04,5466.0,S104A000,C104CC00,SS,-7.0,104,B
...,...,...,...,...,...,...,...,...,...,...
2808,NaN,NaN,NaN,5930.0,Z20JZPR0,E20JD20J,NaN,NaN,20J,new relation
2809,NaN,NaN,NaN,5931.0,Z20KZPR0,E20KD20K,NaN,NaN,20K,new relation
2810,NaN,NaN,NaN,5932.0,Z20LZPR0,E20LD20L,NaN,NaN,20L,new relation
2811,NaN,NaN,NaN,5933.0,Z20MZPR0,E20MD20M,NaN,NaN,20M,new relation


In [8]:
# df_activity_info는 각 activity에 대해 더 상세한 정보가 담겨있는 테이블로, 본 테이블에서 필요한 정보를 가져와 분석에 활용
# LOCATIONTYPE이 O, Z에 해당하는 작업은 후행계획으로 일단 제외
df_activity_info.drop(df_activity_info[(df_activity_info['LOCATIONTYPE'] == 'O') | (df_activity_info['LOCATIONTYPE'] == 'Z')].index, inplace=True)

In [9]:
# df_activity_info에서 날짜에 해당하는 데이터 타입 string -> datetime으로 변환
for col in date_col_list:
    # print(df_activity_info[col])
    df_activity_info[col] = df_activity_info[col].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') if type(x) == str else x )
    df_activity_info[col] = df_activity_info[col].values.astype('datetime64[D]')

# 계획 및 실적 착수/완료일/작업기간 사이의 차이 계산해 저장
temp_list = []
plan_start = df_activity_info['PLANSTARTDATE'].values.astype('datetime64[D]')
actual_start = df_activity_info['ACTUALSTARTDATE'].values.astype('datetime64[D]')
for plan, actual in zip(plan_start, actual_start):
    if np.isnat(plan) == False and np.isnat(actual) == False:
        temp_list.append(np.busday_count(plan, actual))
    else:
        temp_list.append(np.nan)
df_activity_info['PLAN_ACTUAL_START_DIFF'] = temp_list

temp_list = []
plan_start = df_activity_info['PLANFINISHDATE'].values.astype('datetime64[D]')
actual_start = df_activity_info['ACTUALFINISHDATE'].values.astype('datetime64[D]')
for plan, actual in zip(plan_start, actual_start):
    if np.isnat(plan) == False and np.isnat(actual) == False:
        temp_list.append(np.busday_count(plan, actual))
    else:
        temp_list.append(np.nan)
df_activity_info['PLAN_ACTUAL_FINISH_DIFF'] = temp_list

temp_list = []
plan_start = df_activity_info['ACTUALSTARTDATE'].values.astype('datetime64[D]')
actual_start = df_activity_info['ACTUALFINISHDATE'].values.astype('datetime64[D]')
for plan, actual in zip(plan_start, actual_start):
    if np.isnat(plan) == False and np.isnat(actual) == False:
        temp_list.append(np.busday_count(plan, actual))
    else:
        temp_list.append(np.nan)
df_activity_info['ACTUALDURATION'] = temp_list

df_activity_info['PLAN_ACTUAL_DURATION_DIFF'] = df_activity_info['ACTUALDURATION'] - df_activity_info['PLANDURATION']

In [10]:
# df_activity, df_relation의 ACTIVITYCODE에서 각 공정 정보만 추출하여 KEY 생성해 저장
df_activity_info['KEY'] = df_activity_info['FLAGCODE'] + df_activity_info['PROCESSCODE'] + df_activity_info['PROCESSSERNO']

df_activity['KEY'] = df_activity['ACTIVITYCODE'].apply(lambda x: x if x == 'EM/EEM' else(x[0] + x[-4:] if len(x) > 3 else x))
df_activity['KEY'] = df_activity['KEY'].apply(lambda x: x[:2] if (x[:2] == 'ED' or x[:2] =='EP') & (x[-3:] != 'G00' and x[-3:] != 'G01' and x[-3:] != 'S00' and x[-3:] != '000' and x[-3:] != 'M00') else x)

df_relation['PREVKEY'] = df_relation['PREVACTIVITYCODE'].apply(lambda x: x[0] + x[-4:] if len(x) > 3 else x)
df_relation['PREVKEY'] = df_relation['PREVKEY'].apply(lambda x: x[:2] if (x[:2] == 'ED' or x[:2] =='EP') & (x[-3:] != 'G00' and x[-3:] != 'G01' and x[-3:] != 'S00' and x[-3:] != '000' and x[-3:] != 'M00') else x)

df_relation['NEXTKEY'] = df_relation['NEXTACTIVITYCODE'].apply(lambda x: x[0] + x[-4:] if len(x) > 3 else x)
df_relation['NEXTKEY'] = df_relation['NEXTKEY'].apply(lambda x: x[:2] if (x[:2] == 'ED' or x[:2] =='EP') & (x[-3:] != 'G00' and x[-3:] != 'G01' and x[-3:] != 'S00' and x[-3:] != '000' and x[-3:] != 'M00') else x)


In [23]:
# 공정 종류 KEY를 index로 가지는 dataframe 생성
index_list = list(pd.Series(list(set(df_activity['KEY']))).sort_values())
df_process = pd.DataFrame(index=index_list, columns = ['COUNT', 'LOCATIONTYPE',
                                                        'PLAN_DURATION_MEAN', 'PLAN_DURATION_STD',
                                                        'ACTUAL_DURATION_MEAN', 'ACTUAL_DURATION_STD',
                                                        'VOL_MEAN', 'VOL_STD',
                                                        'MH_MEAN', 'MH_STD',
                                                        'PLAN_ACTUAL_START_DIFF_MEAN', 'PLAN_ACTUAL_START_DIFF_STD',
                                                        'PLAN_ACTUAL_FINISH_DIFF_MEAN', 'PLAN_ACTUAL_FINISH_DIFF_STD',
                                                        'PLAN_ACTUAL_DURATION_DIFF_MEAN', 'PLAN_ACTUAL_DURATION_DIFF_STD'])

In [24]:
# df_activity에 KEY로 groupby를 적용하여 각 공정 종류별 통계량을 계산한 후 df_process에 저장
df_process['COUNT'] = df_activity.groupby(by=['KEY']).count().iloc[:,0]
df_process['LOCATIONTYPE'] = df_activity.groupby(by=['KEY']).first()['LOCATIONTYPE']

In [25]:
for i, row in df_activity_info.dropna().groupby(by=['KEY']).mean().iterrows():
    df_process.loc[i, 'PLAN_DURATION_MEAN'] = row['PLANDURATION']
    df_process.loc[i, 'ACTUAL_DURATION_MEAN'] = row['ACTUALDURATION']
    df_process.loc[i, 'VOL_MEAN'] = row['VOL1']
    df_process.loc[i, 'MH_MEAN'] = row['MH1']
    df_process.loc[i, 'PLAN_ACTUAL_START_DIFF_MEAN'] = row['PLAN_ACTUAL_START_DIFF']
    df_process.loc[i, 'PLAN_ACTUAL_FINISH_DIFF_MEAN'] = row['PLAN_ACTUAL_FINISH_DIFF']
    df_process.loc[i, 'PLAN_ACTUAL_DURATION_DIFF_MEAN'] = row['PLAN_ACTUAL_DURATION_DIFF']

for i, row in df_activity_info.dropna().groupby(by=['KEY']).std().iterrows():
    df_process.loc[i, 'PLAN_DURATION_STD'] = row['PLANDURATION']
    df_process.loc[i, 'ACTUAL_DURATION_STD'] = row['ACTUALDURATION']
    df_process.loc[i, 'VOL_STD'] = row['VOL1']
    df_process.loc[i, 'MH_STD'] = row['MH1']
    df_process.loc[i, 'PLAN_ACTUAL_START_DIFF_STD'] = row['PLAN_ACTUAL_START_DIFF']
    df_process.loc[i, 'PLAN_ACTUAL_FINISH_DIFF_STD'] = row['PLAN_ACTUAL_FINISH_DIFF']
    df_process.loc[i, 'PLAN_ACTUAL_DURATION_DIFF_STD'] = row['PLAN_ACTUAL_DURATION_DIFF']

In [15]:
# df_process 확인
df_process

,COUNT,LOCATIONTYPE,PLAN_DURATION_MEAN,PLAN_DURATION_STD,ACTUAL_DURATION_MEAN,ACTUAL_DURATION_STD,VOL_MEAN,VOL_STD,MH_MEAN,MH_STD,PLAN_ACTUAL_START_DIFF_MEAN,PLAN_ACTUAL_START_DIFF_STD,PLAN_ACTUAL_FINISH_DIFF_MEAN,PLAN_ACTUAL_FINISH_DIFF_STD,PLAN_ACTUAL_DURATION_DIFF_MEAN,PLAN_ACTUAL_DURATION_DIFF_STD
AO000,57,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCC00,218,B,17,6.32743,24.973,15.1294,187.775,98.9786,122.179,60.4567,-2.33333,13.2304,5.81982,19.025,7.97297,17.0462
CCP00,218,B,15.766,5.26123,12.6099,11.3734,0,0,4.97518,4.28166,-1.25532,9.61651,-3.87943,14.2956,-3.15603,11.8595
ED,68,New Activity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EDG00,67,E,11.3333,1.1413,17.0455,5.8136,297.212,190.54,375.556,248.743,-2.04545,1.44043,3.71212,5.77732,5.71212,5.50179
EDG01,1,E,12,NaN,12,NaN,214.6,NaN,346,NaN,1,NaN,2,NaN,0,NaN
EDS00,67,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EM/EEM,1,New Activity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EP,205,New Activity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EP000,53,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# (선행 공정 종류 KEY, 후행공정 종류 KEY)를 index로 가지는 dataframe 생성 (MultiIndex가 됨, 인덱스 접근 및 반환은 튜플 형태로 이루어짐)
df_process_relation = pd.DataFrame(index = df_relation.groupby(by=['PREVKEY', 'NEXTKEY']).count().index,
                                   columns = ['COUNT', 'RELATIONTYPE', 'LOCATIONTYPE',
                                              'PREV_PROCESS_PLAN_DURATION','PREV_PROCESS_ACTUAL_DURATION'])

In [17]:
# 마찬가지로 df_relation에 groupby를 적용하여 공정 종류 사이의 연결 관계에 대한 통계량 계산 및 저장
df_process_relation['COUNT'] = df_relation.groupby(by=['PREVKEY', 'NEXTKEY']).count()['NEXTACTIVITYCODE']
df_process_relation['RELATIONTYPE'] = df_relation.groupby(by=['PREVKEY', 'NEXTKEY']).first()['RELATIONTYPE']
df_process_relation['LOCATIONTYPE'] = df_relation.groupby(by=['PREVKEY', 'NEXTKEY']).first()['LOCATIONTYPE']
for idx in df_process_relation.index:
    df_process_relation['PREV_PROCESS_PLAN_DURATION'][idx] = df_process.loc[idx[0],  'PLAN_DURATION_MEAN']
    df_process_relation['PREV_PROCESS_ACTUAL_DURATION'][idx] = df_process.loc[idx[0],  'ACTUAL_DURATION_MEAN']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
# df_process_relation 확인
df_process_relation

COUNT  ... PREV_PROCESS_ACTUAL_DURATION
PREVKEY NEXTKEY         ...                             
AO000   ED           2  ...                          NaN
CCC00   CCP00      218  ...                       24.973
ED      ED          83  ...                          NaN
        F/S          1  ...                          NaN
        L/C          4  ...                          NaN
...                ...  ...                          ...
ZZBR0   WGL00       12  ...                     0.265625
ZZDI0   EDS00       67  ...                      3.53731
        EPM00        4  ...                      3.53731
ZZPR0   ED          45  ...                      1.09302
        EP           7  ...                      1.09302

[67 rows x 5 columns]

In [19]:
# df_process_relation의 선행공정, 후행공정 key 중 df_process에 없는 것이 있는지 확인
temp_list = []
for key in df_relation['PREVKEY']:
    if key not in list(df_process.index):
        temp_list.append(key)
        print(key)
print(len(temp_list), "/", len(df_relation))
print()

# L/C, M/E, F/S 가 없는 것으로 확인 됨, 네트워크 생성과정에서 해당 노드를 자동으로 생성해주기 때문에 큰 문제는 없으나 확인은 필요
temp_list = []
for key in df_relation['NEXTKEY']:
    if key not in list(df_process.index):
        temp_list.append(key)
        print(key)
print(len(temp_list), "/", len(df_relation))

0 / 2813

L/C
M/E
L/C
L/C
L/C
F/S
6 / 2813


In [22]:
# activity를 node로 가지는 그래프 생성
act_network = nx.DiGraph()
for idx, row in df_process.iterrows():
    # df_process의 인덱스, 즉 공정 KEY가 node의 이름이 됨
    act_network.add_node(idx)
    # df_process의 각 칼럼의 내용을 각 node의 attribute로 저장할 수 있음
    for item, col in zip(row, df_process.columns):
        act_network.nodes[idx][col] = item

# relation으로 edge 연결 (현재 node에 없는 activity가 있는 경우, node를 자동으로 생성함)
for idx, row in df_process_relation.iterrows():
    # df_process_relation의 index (선행공정 KEY, 후행공정 KEY)에 해당하는 edge 생성
    act_network.add_edge(idx[0], idx[1])
    # df_process_relation의 각 칼럼의 내용을 각 edge의 attribute로 저장
    for item, col in zip(row, df_process_relation.columns):
        act_network.edges[idx[0], idx[1]][col] = item



In [26]:
# node 접근 테스트
act_network.nodes['PBP00']

{'ACTUAL_DURATION_MEAN': 5.867924528301887,
 'ACTUAL_DURATION_STD': 6.7073672017475,
 'COUNT': 177,
 'LOCATIONTYPE': 'B',
 'MH_MEAN': 98.72264150943393,
 'MH_STD': 64.70953006772542,
 'PLAN_ACTUAL_DURATION_DIFF_MEAN': -0.8427672955974843,
 'PLAN_ACTUAL_DURATION_DIFF_STD': 7.018541980647657,
 'PLAN_ACTUAL_FINISH_DIFF_MEAN': 1.6918238993710693,
 'PLAN_ACTUAL_FINISH_DIFF_STD': 9.02314161625651,
 'PLAN_ACTUAL_START_DIFF_MEAN': 2.050314465408805,
 'PLAN_ACTUAL_START_DIFF_STD': 7.844048572118321,
 'PLAN_DURATION_MEAN': 6.710691823899371,
 'PLAN_DURATION_STD': 0.9232983797761616,
 'VOL_MEAN': 4338.686792452831,
 'VOL_STD': 2440.210715095105}

In [27]:
# edge 접근 테스트
act_network.edges[('PBP00', 'WGL00')]

{'COUNT': 2,
 'LOCATIONTYPE': 'B',
 'PREV_PROCESS_ACTUAL_DURATION': 5.867924528301887,
 'PREV_PROCESS_PLAN_DURATION': 6.710691823899371,
 'RELATIONTYPE': 'FS'}

In [ ]:
# 그래프 시각화를 위해 networkX의 DiGraph 구조를 dot 언어로 변환
# dot 언어는 텍스트로 표현한 그래프 언어

# 그래프 출력을 위해 필요한 특성 DiGraph에 추가
act_network.graph['graph']={'rankdir':'LR'} # 기본 방향을 왼쪽 -> 오른쪽으로 설정
act_network.graph['node']={'shape':'circle'} # node의 기본 모양을 원으로 설정
act_network.graph['edges']={'arrowsize':'4.0'} # edge의 기본 크기 설정

# pydot 패키지의 dot 언어 타입로 변환
A = nx.nx_pydot.to_pydot(act_network)
# print(A)

# dot 파일을 graphviz를 이용해 시각화
src = graphviz.Source(A)
# gv, pdf 파일로 출력
src.render("/content/gdrive/My Drive/Colab Notebooks/DSME Network/process_network11.gv", view=True)

'/content/gdrive/My Drive/Colab Notebooks/DSME Network/process_network11.gv.pdf'

In [ ]:
# 시각화 특성 추가 예시 1
# 아래와 같이 NetworkX의 DiGraph 객체의 node, edge에 시각화에 사용되는 특성을 추가해준 후 dot 언어 변환 -> graphviz로 출력하면 시각화되는 특성을 조정할 수 있음
# 자세한 특성은 다음 document 참고: http://www.graphviz.org/doc/info/attrs.html
for node in act_network.nodes:
    act_network.nodes[node]['fontsize'] = '45'
    act_network.nodes[node]['style'] = 'filled'
    try:
        act_network.nodes[node]['width'] = str(int(np.power(act_network.nodes[node]['COUNT'], 0.4)))
        act_network.nodes[node]['height'] = str(int(np.power(act_network.nodes[node]['COUNT'], 0.4)))
    except:
        act_network.nodes[node]['width'] = '1.0'
        act_network.nodes[node]['height'] = '1.0'
    try:
        if act_network.nodes[node]['LOCATIONTYPE'] == 'B':
            act_network.nodes[node]['color'] = 'orange'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'P':
            act_network.nodes[node]['color'] = 'skyblue'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'E':
            act_network.nodes[node]['color'] = 'green'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'New Activity':
            act_network.nodes[node]['color'] = 'red'
        else:
            pass
    except:
        pass

for edge in act_network.edges:
    act_network.edges[edge]['penwidth'] = str(int(np.power(act_network.edges[edge]['COUNT'], 0.6)))
    try:
        if act_network.edges[edge]['RELATIONTYPE'] == 'SS':
            act_network.edges[edge]['style'] = 'dashed'
    except:
        pass

In [ ]:
# 시각화 특성 추가 예시 2
for node in act_network.nodes:
    act_network.nodes[node]['fontsize'] = '45'
    act_network.nodes[node]['style'] = 'bold'
    try:
        act_network.nodes[node]['label'] = "ACTUAL_DURATION \nMEAN : " + str(np.round(act_network.nodes[node]['ACTUAL_DURATION_MEAN'], 2)) + "\nSTD : " + str(np.round(act_network.nodes[node]['ACTUAL_DURATION_STD'], 2))
    except:
        pass

    try:
        act_network.nodes[node]['width'] = str(int(np.power(act_network.nodes[node]['COUNT'], 0.4)))
        act_network.nodes[node]['height'] = str(int(np.power(act_network.nodes[node]['COUNT'], 0.4)))
    except:
        act_network.nodes[node]['width'] = '1.0'
        act_network.nodes[node]['height'] = '1.0'
    try:
        if act_network.nodes[node]['LOCATIONTYPE'] == 'B':
            act_network.nodes[node]['color'] = 'orange'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'P':
            act_network.nodes[node]['color'] = 'skyblue'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'E':
            act_network.nodes[node]['color'] = 'green'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'New Activity':
            act_network.nodes[node]['color'] = 'red'
        else:
            pass
    except:
        pass

for edge in act_network.edges:
    act_network.edges[edge]['penwidth'] = str(int(np.power(act_network.edges[edge]['COUNT'], 0.6)))
    try:
        if act_network.edges[edge]['RELATIONTYPE'] == 'SS':
            act_network.edges[edge]['style'] = 'dashed'
    except:
        pass

In [ ]:
# 시각화 특성 추가 예시 3
for node in act_network.nodes:
    act_network.nodes[node]['fontsize'] = '40'
    act_network.nodes[node]['style'] = 'bold'
    try:
        act_network.nodes[node]['label'] = "PLN_ACT_DIFF \nSTART : " + str(np.round(act_network.nodes[node]['PLAN_ACTUAL_START_DIFF_MEAN'], 2))\
                                                    + "\nFINISH : " + str(np.round(act_network.nodes[node]['PLAN_ACTUAL_FINISH_DIFF_MEAN'], 2))\
                                                    + "\nDURATION : " + str(np.round(act_network.nodes[node]['PLAN_ACTUAL_DURATION_DIFF_MEAN'], 2))
        if act_network.nodes[node]['PLAN_ACTUAL_DURATION_DIFF_MEAN'] > 0:
            act_network.nodes[node]['fontcolor'] = 'red'
        elif act_network.nodes[node]['PLAN_ACTUAL_DURATION_DIFF_MEAN'] < 0:
            act_network.nodes[node]['fontcolor'] = 'blue'
        else:
            pass
    except:
        pass

    try:
        act_network.nodes[node]['width'] = str(int(np.power(act_network.nodes[node]['COUNT'], 0.4)))
        act_network.nodes[node]['height'] = str(int(np.power(act_network.nodes[node]['COUNT'], 0.4)))
    except:
        act_network.nodes[node]['width'] = '1.0'
        act_network.nodes[node]['height'] = '1.0'
    try:
        if act_network.nodes[node]['LOCATIONTYPE'] == 'B':
            act_network.nodes[node]['color'] = 'orange'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'P':
            act_network.nodes[node]['color'] = 'skyblue'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'E':
            act_network.nodes[node]['color'] = 'green'
        elif act_network.nodes[node]['LOCATIONTYPE'] == 'New Activity':
            act_network.nodes[node]['color'] = 'red'
        else:
            pass
    except:
        pass

for edge in act_network.edges:
    act_network.edges[edge]['penwidth'] = str(int(np.power(act_network.edges[edge]['COUNT'], 0.6)))
    try:
        if act_network.edges[edge]['RELATIONTYPE'] == 'SS':
            act_network.edges[edge]['style'] = 'dashed'
    except:
        pass